<a href="https://colab.research.google.com/github/aboelela924/tensorFlowTutorials/blob/master/BasicClassificationgTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>